In [4]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy.sparse import coo_array
from scipy import sparse
from cdlib import algorithms
from cdlib import evaluation
import sklearn
from utils import *
from distances import *
from consensus import *
from mcl import *
import math
import itertools
import random
import time
from pathlib import Path

In [5]:
cons_name = "v6"

In [63]:
from scipy.sparse import coo_array
from scipy.sparse import csr_array
from mcl import *

# Assumes the elements of the cluster are named as 0-based indices
def v6_consensus(P_list, niter=10, starting_partition=None, verbose=False):
    G = nx.Graph(P_list[0]["graph"])
    n = len(list(G.nodes()))
    k = len(P_list)
    print("Number of edges in G:", len(list(G.edges())))

    t1 = time.time()
    A = nx.to_scipy_sparse_array(G, format="coo")
    t2 = time.time()
    print("Time to get sparse matrix of the graph:", t2-t1)

    nz_rows = A.row 
    nz_cols = A.col
    
    t1 = time.time()
    P_list_asn = []
    c = np.zeros((n,k))
    for i in range(k):
        clust_lst = P_list[i]["partition"]
        clust_asn = clust_lst_to_asn(clust_lst)
        c[:,i] = np.array(clust_asn)
    t2 = time.time()
    print("Time to generate cluster assignment matrix:", t2-t1)
    
    Aw_rows = A.row
    Aw_cols = A.col
    Aw_vals = A.data
    #nz_elems = []
    t1 = time.time()
    for i in range(len(nz_rows)):
        Aw_vals[i] = np.sum( c[nz_rows[i],:] == c[nz_cols[i],:] )
    Aw = csr_array((Aw_vals, (Aw_rows, Aw_cols)), shape=(n, n))
    t2 = time.time()
    print("Time to generate weighted consensus graph:", t2-t1)
    print("Number of non-zeroes in Aw:", Aw.count_nonzero())
    Gw = nx.from_numpy_array(Aw)
    Aw_nnz = Aw.count_nonzero()
    print("Aw_nnz", Aw_nnz)
    
    """
    t1 = time.time()
    bfs_rows = []
    bfs_cols = []
    bfs_vals = []
    for x in range(n):
        bfs_edges = list(nx.bfs_tree(Gw, source=x, depth_limit=2).edges())
        for edge in bfs_edges:
            y = edge[1]
            bfs_rows.append(x)
            bfs_cols.append(y)
            bfs_vals.append(1)
    bfs_neighbors = csr_array((bfs_vals, (bfs_rows, bfs_cols)), shape=(n, n))
    t2 = time.time()
    print("Time to generate bfs neighborhood graph:", t2-t1)
    bfs_neighbors_nnz = bfs_neighbors.count_nonzero()
    print("bfs_neighbors_nnz", bfs_neighbors_nnz)
    """
    
    t1 = time.time()
    #temp = csr_array(Aw)
    Aw = add_self_loops(Aw, 1)
    Aw = normalize(Aw)
    Aw = prune(Aw, 0.001)
    Aw_nnz = Aw.count_nonzero()
    for i in range(2):
        Aw = iterate(Aw, 2, 2)
        Aw = prune(Aw, 0.001)
        Aw = normalize(Aw)
        Aw_nnz = Aw.count_nonzero()
        print("Iteration", i, "Aw_nnz", Aw_nnz)
    t2 = time.time()
    print("Time to simulate MCL:", t2-t1)
    #markov_clusters = get_clusters(Aw)
    #print("Number of Markov clusters:", len(markov_clusters))
    #print(markov_clusters)
    
    
    #R = nx.to_scipy_sparse_array(G, format="csr")
    
    
    A = nx.to_scipy_sparse_array(G, format="csr")
    
    R = A + Aw
    RT = R.transpose()
    R = R + RT
    print("R nnz:", R.count_nonzero())
    for u in range(n):
        row_start = R.indptr[u]
        row_end = R.indptr[u+1]
        for j in range(row_start, row_end):
            v = R.indices[j]
            R.data[j] = np.sum( (c[int(u),:] == c[int(v),:]) )
    #print("R nnz:", R.count_nonzero())
    
    t1 = time.time()
    refined_partition = None
    if starting_partition:
        refined_partition = list(starting_partition)
    else:
        refined_partition = []
        for i in range(n):
            refined_partition.append([str(i)])
    
    refined_partition_map = clust_lst_to_map(refined_partition)
    items = list(refined_partition_map.keys())
    t2 = time.time()
    print("Time to initialize:", t2-t1)
    
    sparseMatrixMiss = 0
    sparseMatrixHit = 0
    
    tSearch = 0
    tUpdate = 0
    tMovement = 0
    tSparseMatLookup = 0
    count = 0
    it = 1
    last_valid = np.zeros(n)
    last_deltaS = np.zeros(n)
    while(it <= niter):
        print("Iteration:", it)
        potential_moves = {
            "from": np.arange(n),
            "to": np.arange(n),
            "attractor": np.arange(n),
            "deltaS": np.zeros(n),
            "valid": np.zeros(n)
        }
        for u in range(n):
            row_start = A.indptr[u]
            row_end = A.indptr[u+1]
            for j in range(row_start, row_end):
                v = A.indices[j]
                w = A.data[j]
                
                t1 = time.time()
                
                a = refined_partition_map[str(u)]
                b = refined_partition_map[str(v)]
                
                Mua = 0
                Mub = 0
                for elem in refined_partition[a]:
                    if str(elem) != str(u):
                        t3 = time.time()
                        
                        r = 0
                        
                        #r = R[int(u),int(elem)]
                        
                        R_row_start = R.indptr[u]
                        R_row_end = R.indptr[u+1]
                        low = R_row_start
                        hi = R_row_end - 1
                        target = -1
                        while low <= hi:
                            mid = ( low + hi ) // 2
                            if R.indices[mid] < int(elem):
                                low = mid + 1
                            elif R.indices[mid] > int(elem):
                                hi = mid - 1
                            else:
                                target = mid
                                break
                        if target != -1:
                            r = R.data[target]
    
                        t4 = time.time()
                        tSparseMatLookup += t4-t3
                        
                        if (r == 0):
                            Mua = Mua + (k - 2 * np.sum( (c[int(u),:] == c[int(elem),:]) ) )
                            sparseMatrixMiss = sparseMatrixMiss + 1
                            #print("Sparse matrix lookup miss:", sparseMatrixMiss)
                        else:
                            Mua = Mua + (k - 2 * r )
                            sparseMatrixHit = sparseMatrixHit + 1
                            #print("Sparse matrix lookup hit:", sparseMatrixHit)
                for elem in refined_partition[b]:    
                    if str(elem) != str(u):
                        t3 = time.time()
                        
                        r = 0
                        
                        #r = R[int(u),int(elem)]
                        
                        R_row_start = R.indptr[u]
                        R_row_end = R.indptr[u+1]
                        low = R_row_start
                        hi = R_row_end - 1
                        target = -1
                        while low <= hi:
                            mid = ( low + hi ) // 2
                            if R.indices[mid] < int(elem):
                                low = mid + 1
                            elif R.indices[mid] > int(elem):
                                hi = mid - 1
                            else:
                                target = mid
                                break
                        if target != -1:
                            r = R.data[target]
                        
                        t4 = time.time()
                        tSparseMatLookup += t4-t3
                              
                        if (r == 0):
                            Mub = Mub + (k - 2 * np.sum( (c[int(u),:] == c[int(elem),:]) ) )
                            sparseMatrixMiss = sparseMatrixMiss + 1
                            #print("Sparse matrix lookup miss:", sparseMatrixMiss)
                        else:
                            Mub = Mub + (k - 2 * r )
                            sparseMatrixHit = sparseMatrixHit + 1
                            #print("Sparse matrix lookup hit:", sparseMatrixHit)
                            
                deltaS = Mub - Mua
                t2 = time.time()
                tSearch = tSearch + t2-t1
                
                if (deltaS is not None) and (deltaS < 0) and (a != b):
                    potential_moves["from"][u] = a
                    potential_moves["to"][u] = b
                    potential_moves["attractor"][u] = v
                    potential_moves["deltaS"][u] = deltaS
                    potential_moves["valid"][u] = 1
        
        t1 = time.time()
        for u in range(n):
            v = potential_moves["attractor"][u]
            if (potential_moves["valid"][u] == 1) and (potential_moves["attractor"][v] == u) and (potential_moves["valid"][v] == 1):
                # Question mark
                if potential_moves["deltaS"][u] < potential_moves["deltaS"][v]:
                    potential_moves["valid"][v] = 0
                    potential_moves["deltaS"][v] = 0
                else:
                    potential_moves["valid"][u] = 0
                    potential_moves["deltaS"][u] = 0
        t2 = time.time()
        tSearch = tSearch + (t2 - t1)
        
        flag = False
        if np.sum( (potential_moves["valid"] != last_valid) ) == 0:
            # Same set of elements are being moved
            if np.sum(potential_moves["deltaS"]) < np.sum(last_deltaS):
                last_valid = potential_moves["valid"]
                last_deltaS = potential_moves["deltaS"]
                flag = True
        else:
            last_valid = potential_moves["valid"]
            last_deltaS = potential_moves["deltaS"]
            flag = True
        
        if flag == True:
            for u in range(n):
                if potential_moves["valid"][u] == True:
                    a = potential_moves["from"][u]
                    b = potential_moves["to"][u]

                    t1 = time.time()
                    if verbose:
                        print("---")
                        print("Iteration:", it, "Move Count:", count+1, ">> results in deltaS", potential_moves["deltaS"][u])
                        print("Move:", u)
                        print("From partition", a, ":", refined_partition[a])
                        print("To partition", b, ":", refined_partition[b])
                        print("---")
                    refined_partition[a].remove(str(u))
                    refined_partition[b].append(str(u))
                    refined_partition_map[str(u)] = b
                    t2 = time.time()
                    tUpdate = tUpdate + (t2-t1)

                    count = count + 1
        
        print("Sparse matrix lookup miss:", sparseMatrixMiss)
        print("Sparse matrix lookup hit:", sparseMatrixHit)
        
        if flag == False:
            break
        
        it = it + 1
    print("Time to lookup entries in sparse R:", tSparseMatLookup)
    print("Time to search moves:", tSearch)
    print("Time to update M:", tUpdate)
    
    t1 = time.time()
    empty_clusters = []
    for i in range(len(refined_partition)):
        if len(refined_partition[i]) == 0:
            empty_clusters.append(i)
            
    empty_clusters.sort(reverse=True)
    for e in empty_clusters:
        del refined_partition[e]
    t2 = time.time()
    print("Time to delete empty partitions:", t2-t1)
    
    Gw = nx.from_scipy_sparse_array(Aw)
    return {"graph": nx.Graph(Gw), "partition": list(refined_partition)}

# n=200

In [37]:
n = 200
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [64]:
import random

n = 200
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v6_consensus(P_list, niter=100, starting_partition=None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                #write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)

LFR/n200/LFR_n200_mu01_gamma30_beta11.mtx
Number of edges in G: 993
Time to get sparse matrix of the graph: 0.0024111270904541016
Time to generate cluster assignment matrix: 0.0029020309448242188
Time to generate weighted consensus graph: 0.008874177932739258
Number of non-zeroes in Aw: 1924
Aw_nnz 1924
Iteration 0 Aw_nnz 3740
Iteration 1 Aw_nnz 3168
Iteration 2 Aw_nnz 1725
Time to simulate MCL: 0.01769089698791504
R nnz: 2929
Time to initialize: 5.698204040527344e-05
Iteration: 1
Sparse matrix lookup miss: 0
Sparse matrix lookup hit: 1862
Iteration: 2
Sparse matrix lookup miss: 4089
Sparse matrix lookup hit: 16493
Iteration: 3
Sparse matrix lookup miss: 18583
Sparse matrix lookup hit: 57141
Iteration: 4
Sparse matrix lookup miss: 39281
Sparse matrix lookup hit: 108021
Iteration: 5
Sparse matrix lookup miss: 61318
Sparse matrix lookup hit: 160868
Iteration: 6
Sparse matrix lookup miss: 83355
Sparse matrix lookup hit: 213715
Time to lookup entries in sparse R: 3.1922576427459717
Time to

# n=1000

In [10]:
n = 1000
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [12]:
import random

n = 1000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v6_consensus(P_list, niter=1000, starting_partition=None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                #write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)
                

LFR/n1000/LFR_n1000_mu01_gamma30_beta11.mtx
Number of edges in G: 9609
Time to get sparse matrix of the graph: 0.019747495651245117
Time to generate cluster assignment matrix: 0.01180720329284668
Time to generate weighted consensus graph: 0.08393669128417969
Number of non-zeroes in Aw: 17503
Aw_nnz 17503
Iteration 0 Aw_nnz 15694
Iteration 1 Aw_nnz 31425
Iteration 2 Aw_nnz 29398
Iteration 3 Aw_nnz 18943
Iteration 4 Aw_nnz 10775
Iteration 5 Aw_nnz 5725
Iteration 6 Aw_nnz 3375
Iteration 7 Aw_nnz 2151
Iteration 8 Aw_nnz 1580
Iteration 9 Aw_nnz 1184
Time to simulate MCL: 0.14886713027954102
Time to initialize: 0.0002689361572265625
Time to search moves: 0.5758402347564697
Time to update M: 0.0005061626434326172
Time to delete empty partitions: 0.00016689300537109375
mu 1 , number of clusters 44
Time: 1.2675745487213135
LFR/n1000/LFR_n1000_mu02_gamma30_beta11.mtx
Number of edges in G: 9791
Time to get sparse matrix of the graph: 0.12459754943847656
Time to generate cluster assignment matrix:

# n=5000

In [13]:
n = 5000
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [14]:
import random

n = 5000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v6_consensus(P_list, niter=1000, starting_partition = None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                #write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)
                

LFR/n5000/LFR_n5000_mu01_gamma30_beta11.mtx
Number of edges in G: 48950
Time to get sparse matrix of the graph: 0.37200188636779785
Time to generate cluster assignment matrix: 0.0628213882446289
Time to generate weighted consensus graph: 0.4245913028717041
Number of non-zeroes in Aw: 88672
Aw_nnz 88672
Iteration 0 Aw_nnz 79927
Iteration 1 Aw_nnz 149875
Iteration 2 Aw_nnz 135620
Iteration 3 Aw_nnz 94563
Iteration 4 Aw_nnz 53845
Iteration 5 Aw_nnz 30113
Iteration 6 Aw_nnz 16682
Iteration 7 Aw_nnz 10252
Iteration 8 Aw_nnz 7831
Iteration 9 Aw_nnz 6600
Time to simulate MCL: 0.7736890316009521
Time to initialize: 0.0013642311096191406
Time to search moves: 4.242081642150879
Time to update M: 0.002650737762451172
Time to delete empty partitions: 0.0007393360137939453
mu 1 , number of clusters 243
Time: 7.749377727508545
LFR/n5000/LFR_n5000_mu02_gamma30_beta11.mtx
Number of edges in G: 50068
Time to get sparse matrix of the graph: 0.3930835723876953
Time to generate cluster assignment matrix: 